In [ ]:
import os
from config import *
def getAverage(array):
    sum = 0
    for x in array:
        sum += float(x)
    avg = sum / len(array)
    return avg

def getMax(array):
    max = 0
    for x in array:
        if float(x) > max:
            max = float(x)
    return max

def getMin(array):
    min = float(array[0])
    for x in array:
        if float(x) < min:
            min = float(x)
    return min

def getImpactPercent(val, ref):
    return 100*((val/ref) -1)

## Training Data Results

In [ ]:

##NOTE: Dirs must follow training_*/base_*/run_*

BASEDIR= RESULTS_FOLDER + "results_training/"
outfile = open("results_training.log", "w") # change to a
outfile.write("DVFS\t")
for c in range(6):
    outfile.write("b_C"+str(c)+"_Instr\t")
    outfile.write("b_C"+str(c)+"_Access\t")
    outfile.write("b_C"+str(c)+"_Misses\t")
outfile.write("b_Eff\t")
for c in range(6):
    outfile.write("a_C"+str(c)+"_Instr\t")
    outfile.write("a_C"+str(c)+"_Access\t")
    outfile.write("a_C"+str(c)+"_Misses\t")
outfile.write("a_Eff\n")

#iterating over all training file results
for training_dir in sorted(os.listdir(BASEDIR)):
    t = os.path.join(BASEDIR, training_dir)
    if os.path.isdir(t):
        #iterating over each base mapping
        for base_dir in sorted(os.listdir(t)):  
            b = os.path.join(t, base_dir) 
            if os.path.isdir(b): 
                #iterating over each run
                for run_dir in  sorted(os.listdir(b)):
                    run = os.path.join(b, run_dir)
                    #iterating over result files
                    files = sorted(os.listdir(run))
                    #let the magic happen here
                    baseline =os.path.join(run, files[0]) 
                    dvfs =os.path.join(run, files[1])
                    variants = os.path.join(run, files[2])

                    var_file = open(variants, "r")
                    var_data = var_file.readlines()
                    base_file = open(baseline, "r")
                    base_data = base_file.readlines()
                    dvfs_file = open(dvfs, "r")
                    dvfs_data = dvfs_file.readlines()
                
                    for line in var_data:
                        newline = "0\t" + base_data[0][:-1] + "\t" + line
                        outfile.write(newline)
                        newline = "1\t" + dvfs_data[0][:-1] + "\t" + line
                        outfile.write(newline)
                    var_file.close()
                    base_file.close()
                    dvfs_file.close()
outfile.close()
print("Training results generated")

## Motivation/Run-time Results

In [ ]:
BASEDIR=RESULTS_FOLDER + "results_motiv/"
outfile = open("results_runtime.log", "w")

policyAverages = []
policyMin = []
policyMax = []
policies = []
#iterating over all  results folder
for result_dir in sorted(os.listdir(BASEDIR)):
    t = os.path.join(BASEDIR, result_dir)
    if os.path.isdir(t):
        policies.append(result_dir[:result_dir.find("_")])
        avgs = []
        #iterating over each run mapping
        for run_dir in sorted(os.listdir(t)):
            run = os.path.join(t, run_dir)
            #iterating over result files
            files = sorted(os.listdir(run))
            eff_file = open(os.path.join(run, files[0]), "r")
            eff_data = eff_file.readlines()
            avgs.append(getAverage(eff_data))
            eff_file.close()
        policyAverages.append(getAverage(avgs))
        policyMax.append(getMax(avgs))
        policyMin.append(getMin(avgs))
        
print(policies)
print(policyAverages)
print(policyMax)
print(policyMin)


impact_eff_avg = []
impact_eff_mins = []
impact_eff_maxs = []
for x in range(1,len(policies)):
    impact_eff_avg.append(getImpactPercent(policyAverages[x],
                                       policyAverages[0]))
    impact_eff_mins.append(getImpactPercent(policyMin[x],
                                       policyMin[0]))
    impact_eff_maxs.append(getImpactPercent(policyMax[x],
                                       policyMax[0]))


print(impact_eff_mins)

outfile.close()

print("Runt-time results generated")
